In [1]:
# Imports
from dask_jobqueue import SLURMCluster
from distributed import Client
from run_cocycles_discrete import run

In [2]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="16GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10095",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
    #job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=100)
client = Client(cluster)

In [3]:
# Submitting jobs
n = 500
ntrial = 20
corr = True
affine = True
futures = []
for seed in range(ntrial):
    f0 = client.submit(run,n = n, seed = seed, wrong_order = False, corr = corr, affine = affine)
    f1 = client.submit(run,n = n, seed = seed, wrong_order = True, corr = corr, affine = affine)
    futures += [f0,f1]

In [4]:
futures

[<Future: pending, key: run-0ff147ff7fa22f827a4e1ef87a7b4ab4>,
 <Future: pending, key: run-4b0b845a9a469427b8164f4ef7924cd5>,
 <Future: pending, key: run-27c820746d6333d2aa3d6904ffa68c9d>,
 <Future: pending, key: run-514d49a255fdbf1c342163b44c273807>,
 <Future: pending, key: run-b886c741e39a88cdb079a1a491ff7eb9>,
 <Future: pending, key: run-1e645df74c285d83baf76dc3fd938bc3>,
 <Future: pending, key: run-a160b42d7bf6fef76c04d07a3a744166>,
 <Future: pending, key: run-c8fa116f9fff83fa251630308b441919>,
 <Future: pending, key: run-e1a27a6bef0d43fa098ebaa3960bc264>,
 <Future: pending, key: run-68ae92ae2c044bc5ba89f1386ee726e4>,
 <Future: pending, key: run-1b84ee6229a704ebfbac50040780f90c>,
 <Future: pending, key: run-4b35165e4ed0d51a6dee80235834bcfb>,
 <Future: pending, key: run-42835facd26ad6bfe2abba3c6952f79a>,
 <Future: pending, key: run-ad815f85f51403c9fa440fe3c439dfd9>,
 <Future: pending, key: run-3db926f67c53143f1e3d978503709501>,
 <Future: pending, key: run-40f90674fed20046d7394c743c2

In [ ]:
# Gettaing results
results = client.gather(futures)

In [ ]:
# Closing client
client.close()
cluster.close()

In [ ]:
import torch
torch.save(f = "nfcocycle_cv_results_trials={0}_n={1}_corr={2}_affine={3}_new.pt".format(ntrial,n,corr,affine), obj = results)

In [ ]:
results

In [ ]:
rmse_correct = []
rmse_wrong = []
for i in range(len(results)):
    if not i % 2:
        rmse_correct.append(results[i]['RMSE20'])
    else:
        rmse_wrong.append(results[i]['RMSE20'])

print(torch.tensor(rmse_correct).nanmedian())
print(torch.tensor(rmse_wrong).nanmedian())